In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import re
import requests
import pickle
import time

I need more features for the data to build a more robust model. <br>
Obtaining additional movie info from MovieLens datasets from Kaggle for: IMDb IDs, MovieLens rating, IMDb rating(need to scrap from IMDb with IMDb IDs)

## Contents:
1. Accessing other dataset for IMDb ID
2. Merge dataset with Ebert dataset
3. Obtaining IMDb ID and web-scrape from IMDb.com
4. Merge Ebert df and IMDb df
5. Data Cleaning

## 1. Accessing other dataset for IMDb ID

In [2]:
movies_df = pd.read_csv('./data/movie.csv')
links_df = pd.read_csv('./data/link.csv')
ratings_df = pd.read_csv('./data/rating.csv')
ratings_df = ratings_df.groupby(['movieId']).mean().reset_index()

movielens_df = movies_df.merge(ratings_df, how='left', on='movieId')
movielens_df = movielens_df.merge(links_df, how='left', on='movieId')

movielens_df.drop(columns=['movieId','genres','userId','tmdbId'], inplace=True)

In [3]:
movielens_df['year'] = [''.join(filter(lambda i: i.isdigit(), title_year)) for title_year in movielens_df.title]
movielens_df['year'] = [year[-4:] for year in movielens_df.year.astype(str)]

movielens_df['title'] = movielens_df['title'].str.replace(r'\(.*\)','', regex=True)
movielens_df['title'] = movielens_df['title'].str.rstrip()
movielens_df = movielens_df.rename(columns={'title': 'TITLE'})

In [4]:
movielens_df.sample(10)

,TITLE,rating,imdbId,year
24758,Bloody Birthday,2.500000,82084,1981
10250,Crippled Masters,2.333333,122029,1979
15967,One Million B.C.,3.062500,32871,1940
17551,Exodus,3.500000,1091616,2007
15301,"Velvet Vampire, The",2.000000,67929,1971
21626,King of Texas,4.000000,282659,2002
26451,Fantomas vs. Scotland Yard,3.000000,60400,1967
18252,Another Cinderella Story,3.073529,1071358,2008
7781,Father Hood,2.074074,106877,1993
5203,3:10 to Yuma,3.637746,50086,1957


## 2. Merge dataset with Ebert dataset

In [5]:
# Accessing ebert_df
ebert_df = pd.read_pickle('./data/ebert_df_cleaned.pickle')
ebert_df['YEAR'] = ebert_df.YEAR.astype(str)
ebert_df.head(10)

,TITLE,YEAR,EBERT_RATING,MPAA,RUNTIME,GENRE,sub-genre,link
0,Computer Chess,2013,2.0,NR,91 minutes,Comedy,N,https://www.rogerebert.com/reviews/computer-ch...
1,At Any Price,2012,4.0,R,105 minutes,Drama,N,https://www.rogerebert.com/reviews/at-any-pric...
2,Blancanieves,2012,4.0,PG-13,104 minutes,Fantasy,Drama,https://www.rogerebert.com/reviews/blancanieve...
3,To the Wonder,2013,3.5,R,112 minutes,Romance,Drama,https://www.rogerebert.com/reviews/to-the-wond...
4,From Up on Poppy Hill,2013,2.5,PG,91 minutes,Drama,Animation,https://www.rogerebert.com/reviews/from-up-on-...
5,The Host,2013,2.5,PG-13,125 minutes,Thriller,Science Fiction,https://www.rogerebert.com/reviews/the-host-2013
6,Ginger and Rosa,2013,3.0,PG-13,89 minutes,Drama,N,https://www.rogerebert.com/reviews/ginger-and-...
7,On the Road,2013,2.0,R,137 minutes,Drama,Adventure,https://www.rogerebert.com/reviews/on-the-road...
8,Future Weather,2012,3.5,NR,100 minutes,Drama,N,https://www.rogerebert.com/reviews/future-weat...
9,Side Effects,2013,3.5,R,106 minutes,Thriller,Drama,https://www.rogerebert.com/reviews/side-effect...


In [6]:
# Merging ebert_df with movielens_df
ebert_imdb = ebert_df.merge(movielens_df, how='left', left_on=['TITLE','YEAR'], right_on=['TITLE', 'year'])
ebert_imdb.head(10)

,TITLE,YEAR,EBERT_RATING,MPAA,RUNTIME,GENRE,sub-genre,link,rating,imdbId,year
0,Computer Chess,2013,2.0,NR,91 minutes,Comedy,N,https://www.rogerebert.com/reviews/computer-ch...,3.214286,2007360.0,2013
1,At Any Price,2012,4.0,R,105 minutes,Drama,N,https://www.rogerebert.com/reviews/at-any-pric...,3.125000,1937449.0,2012
2,Blancanieves,2012,4.0,PG-13,104 minutes,Fantasy,Drama,https://www.rogerebert.com/reviews/blancanieve...,NaN,NaN,NaN
3,To the Wonder,2013,3.5,R,112 minutes,Romance,Drama,https://www.rogerebert.com/reviews/to-the-wond...,3.063830,1595656.0,2013
4,From Up on Poppy Hill,2013,2.5,PG,91 minutes,Drama,Animation,https://www.rogerebert.com/reviews/from-up-on-...,NaN,NaN,NaN
5,The Host,2013,2.5,PG-13,125 minutes,Thriller,Science Fiction,https://www.rogerebert.com/reviews/the-host-2013,NaN,NaN,NaN
6,Ginger and Rosa,2013,3.0,PG-13,89 minutes,Drama,N,https://www.rogerebert.com/reviews/ginger-and-...,NaN,NaN,NaN
7,On the Road,2013,2.0,R,137 minutes,Drama,Adventure,https://www.rogerebert.com/reviews/on-the-road...,NaN,NaN,NaN
8,Future Weather,2012,3.5,NR,100 minutes,Drama,N,https://www.rogerebert.com/reviews/future-weat...,3.250000,1701215.0,2012
9,Side Effects,2013,3.5,R,106 minutes,Thriller,Drama,https://www.rogerebert.com/reviews/side-effect...,3.642730,2053463.0,2013


In [7]:
# Checking for NaN
ebert_imdb.isna().sum()

TITLE              0
YEAR               0
EBERT_RATING       0
MPAA               0
RUNTIME            0
GENRE              0
sub-genre          0
link               0
rating          3436
imdbId          3435
year            3435
dtype: int64

In [8]:
# Dropping rows without imdbId and movielens rating
ebert_imdb.dropna(subset=['imdbId','rating'],how='any',inplace=True)

## 3. Obtaining IMDb ID and web-scrape from IMDb.com

In [9]:
# Standardize IMDb ID for url access
ebert_imdb['imdbId'] = ebert_imdb.imdbId.astype(int).astype(str)
ebert_imdb['imdbId'] = [str(item).zfill(7) for item in ebert_imdb.imdbId]

In [10]:
def get_imdb_data(movie_path):
    """
    Scrapes IMDb.com for users' movie rating

    """
    
    pause_time = np.random.randint(0,1)
    time.sleep(pause_time)
    url = 'http://www.imdb.com/title/tt' + movie_path
    r = requests.get(url)
    s = bs(r.text, 'html5lib')
    
    imdb_data = {}
    
    imdb_data['imdbId'] = movie_path
    #title
    imdb_data['title'] = s.find('title').text.split(' (')[0]
    
    #imdb_rating
    try:
        imdb_data['imdb_rating'] = s.find(class_='ratingValue').text.split('/')[0].strip('\n')
    except:
        imdb_data['imdb_rating'] = np.nan
    
    detail = s.find(attrs={'id':'titleDetails'})
    #budget
    try:
        imdb_data['budget'] = detail.find('h4', text="Budget:").next_sibling.strip()
    except:
        imdb_data['budget'] = np.nan
    
    #opening gross
    try:
        imdb_data['opening'] = detail.find('h4', text="Opening Weekend USA:").next_sibling.strip()
    except:
        imdb_data['opening'] = np.nan
    
    #usa gross
    try:
        imdb_data['gross_USA'] = detail.find('h4', text="Gross USA:").next_sibling.strip()
    except:
        imdb_data['gross_USA'] = np.nan
        
    #total gross
    try:
        imdb_data['totalgross'] = detail.find('h4', text="Cumulative Worldwide Gross:").next_sibling.strip()
    except:
        imdb_data['totalgross'] = np.nan
    
    
    return imdb_data

In [11]:
# Get IMDb data
imdb_data = []
for movie_path in ebert_imdb['imdbId']:
    imdb_data.append(get_imdb_data(movie_path))
# Save as pickle just in case
with open('./data/imdb_data.pickle', 'wb') as w:
    pickle.dump(imdb_data, w)

In [12]:
imdb_data = pd.read_pickle('./data/imdb_data.pickle')

In [13]:
# Convert list of dictionaries to DataFrame
imdb_data_df = pd.DataFrame(imdb_data)
imdb_data_df

,imdbId,title,imdb_rating,budget,opening,gross_USA,totalgross
0,2007360,Computer Chess,6.3,NaN,"$9,683,","$102,041","$127,852"
1,1937449,At Any Price,5.6,NaN,"$16,095,","$380,594","$487,965"
2,1595656,To the Wonder,5.9,NaN,"$116,551,","$587,615","$2,801,166"
3,1701215,Future Weather,6.1,NaN,NaN,NaN,NaN
4,2053463,Side Effects,7.1,"$30,000,000","$9,303,145,","$32,172,757","$63,414,135"
...,...,...,...,...,...,...,...
3323,0061610,"Easy Come, Easy Go",5.3,"$2,000,000",NaN,NaN,NaN
3324,0061796,Hurry Sundown,5.8,"$4,000,000",NaN,NaN,NaN
3325,0061452,Casino Royale,5.1,"$12,000,000",NaN,NaN,"$2,324"
3326,0061770,Hombre,7.4,"$5,860,000",NaN,NaN,NaN


In [14]:
imdb_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3328 entries, 0 to 3327
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   imdbId       3328 non-null   object
 1   title        3328 non-null   object
 2   imdb_rating  3328 non-null   object
 3   budget       2481 non-null   object
 4   opening      2798 non-null   object
 5   gross_USA    3022 non-null   object
 6   totalgross   3044 non-null   object
dtypes: object(7)
memory usage: 182.1+ KB


## 4. Merge Ebert df and IMDb df

In [15]:
merged_df = ebert_imdb.merge(imdb_data_df, how='left', left_on='imdbId', right_on='imdbId')
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3328 entries, 0 to 3327
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   TITLE         3328 non-null   object 
 1   YEAR          3328 non-null   object 
 2   EBERT_RATING  3328 non-null   float64
 3   MPAA          3328 non-null   object 
 4   RUNTIME       3328 non-null   object 
 5   GENRE         3328 non-null   object 
 6   sub-genre     3328 non-null   object 
 7   link          3328 non-null   object 
 8   rating        3328 non-null   float64
 9   imdbId        3328 non-null   object 
 10  year          3328 non-null   object 
 11  title         3328 non-null   object 
 12  imdb_rating   3328 non-null   object 
 13  budget        2481 non-null   object 
 14  opening       2798 non-null   object 
 15  gross_USA     3022 non-null   object 
 16  totalgross    3044 non-null   object 
dtypes: float64(2), object(15)
memory usage: 468.0+ KB


In [16]:
merged_df.isna().sum()

TITLE             0
YEAR              0
EBERT_RATING      0
MPAA              0
RUNTIME           0
GENRE             0
sub-genre         0
link              0
rating            0
imdbId            0
year              0
title             0
imdb_rating       0
budget          847
opening         530
gross_USA       306
totalgross      284
dtype: int64

In [17]:
# Dropping NaN and unneccessary columns
merged_df.dropna(how='any', inplace=True)
merged_df.drop(columns=['link', 'imdbId', 'year', 'title'], inplace=True)
merged_df.shape

(2191, 13)

In [18]:
# saving df to pickle for cleaning
merged_df.to_pickle("./data/merged_df_not_cleaned.pickle")

## 5. Data Cleaning

In [1]:
# Accessing the not cleaned merged df
df = pd.read_pickle('./data/merged_df_not_cleaned.pickle')
df.reset_index(inplace=True, drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2191 entries, 0 to 2190
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   TITLE         2191 non-null   object 
 1   YEAR          2191 non-null   object 
 2   EBERT_RATING  2191 non-null   float64
 3   MPAA          2191 non-null   object 
 4   RUNTIME       2191 non-null   object 
 5   GENRE         2191 non-null   object 
 6   sub-genre     2191 non-null   object 
 7   rating        2191 non-null   float64
 8   imdb_rating   2191 non-null   object 
 9   budget        2191 non-null   object 
 10  opening       2191 non-null   object 
 11  gross_USA     2191 non-null   object 
 12  totalgross    2191 non-null   object 
dtypes: float64(2), object(11)
memory usage: 222.6+ KB


There are some inconsistency of datatype for some columns: YEAR, RUNTIME, IMDB_rating, budget, opening, gross_USA, and totalgross.

In [2]:
df.sample(5)

,TITLE,YEAR,EBERT_RATING,MPAA,RUNTIME,GENRE,sub-genre,rating,imdb_rating,budget,opening,gross_USA,totalgross
445,Mad Money,2008,1.5,PG-13,104 minutes,Crime,Comedy,2.795620,5.9,"$22,000,000","$7,736,452,","$20,668,843","$26,412,163"
533,Peaceful Warrior,2006,2.5,PG-13,120 minutes,Romance,Indie,3.520588,7.3,"$10,000,000","$80,602,","$3,960,414","$4,326,927"
1270,Hurlyburly,1998,3.0,R,122 minutes,Indie,Drama,2.959520,5.9,"$15,000,000","$164,826,","$1,798,862","$1,808,004"
292,Looking for Eric,2009,2.0,PG-13,119 minutes,Indie,Foreign,3.563380,7.2,"GBP4,000,000","$6,743,","$55,804","$11,650,726"
1077,Just Visiting,2001,3.0,PG-13,88 minutes,Science Fiction,Fantasy,2.782609,5.8,"$35,000,000","$2,272,489,","$4,781,539","$16,176,732"


In [3]:
# change column datatypes, get rid of unneccessary characters
df['YEAR'] = df.YEAR.astype(int)
df['RUNTIME'] = [x.replace(' minutes','') for x in df.RUNTIME]
df['RUNTIME'] = df.RUNTIME.astype(int)
df['imdb_rating'] = df.imdb_rating.astype(float)

df['budget'] = df.budget.str.replace(r'[^(\d)]','',regex=True).astype(int)
df['opening'] = df.opening.str.replace(r'[^(\d)]','',regex=True).astype(int)
df['gross_USA'] = df.gross_USA.str.replace(r'[^(\d)]','',regex=True).astype(int)
df['totalgross'] = df.totalgross.str.replace(r'[^(\d)]','',regex=True).astype(int)

# rename column names to be more consistent
df.rename(columns={
    "sub-genre":"SUB_GENRE",
    "rating":"MOVIELENS_RATING", 
    "imdb_rating":"IMDB_RATING", 
    "budget":"BUDGET", 
    "opening":"OPENING_GROSS",
    "gross_USA":"DOMESTIC_GROSS",
    "totalgross":"WORLDWIDE_GROSS"}, inplace=True)

In [4]:
# Sanity check
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2191 entries, 0 to 2190
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TITLE             2191 non-null   object 
 1   YEAR              2191 non-null   int64  
 2   EBERT_RATING      2191 non-null   float64
 3   MPAA              2191 non-null   object 
 4   RUNTIME           2191 non-null   int64  
 5   GENRE             2191 non-null   object 
 6   SUB_GENRE         2191 non-null   object 
 7   MOVIELENS_RATING  2191 non-null   float64
 8   IMDB_RATING       2191 non-null   float64
 9   BUDGET            2191 non-null   int64  
 10  OPENING_GROSS     2191 non-null   int64  
 11  DOMESTIC_GROSS    2191 non-null   int64  
 12  WORLDWIDE_GROSS   2191 non-null   int64  
dtypes: float64(3), int64(6), object(4)
memory usage: 222.6+ KB


In [5]:
# Sanity check
df.head(10)

,TITLE,YEAR,EBERT_RATING,MPAA,RUNTIME,GENRE,SUB_GENRE,MOVIELENS_RATING,IMDB_RATING,BUDGET,OPENING_GROSS,DOMESTIC_GROSS,WORLDWIDE_GROSS
0,Side Effects,2013,3.5,R,106,Thriller,Drama,3.642730,7.1,30000000,9303145,32172757,63414135
1,Quartet,2012,2.5,PG-13,98,Romance,Foreign,3.451807,6.8,11000000,47122,18390117,59520298
2,Playing for Keeps,2012,2.0,PG-13,106,Romance,Drama,2.939024,5.7,35000000,5750288,13103272,30962335
3,Veronika Voss,1982,4.0,R,105,Drama,Crime,3.825658,7.8,2600000,11623,8144,8144
4,Killing Them Softly,2012,2.0,R,104,Thriller,Crime,3.225352,6.2,15000000,6812900,15026056,37930465
5,Red Dawn,2012,1.5,PG-13,114,Thriller,Action,2.697248,5.4,65000000,14276668,44806783,50950296
6,Rise of the Guardians,2012,3.0,PG,97,Animation,Adventure,3.600000,7.3,145000000,23773465,103412758,306941670
7,Hitchcock,2012,3.0,R,98,Drama,N,3.360577,6.8,15700000,287715,6008677,27039669
8,Life of Pi,2012,4.0,PG,127,Drama,Adventure,3.791823,7.9,120000000,22451514,124987023,609016565
9,Silver Linings Playbook,2012,3.5,R,122,Foreign,Drama,3.851326,7.7,21000000,443003,132092958,236412453


Data is now cleaned!

In [6]:
# Save as pickle
df.to_pickle('./data/cleaned_df.pickle')